In [2]:
# download movie lens 100k dataset

# !wget http://files.grouplens.org/datasets/movielens/ml-100k.zip
# !unzip ml-100k.zip
import pandas as pd
import numpy as np
ratings = pd.read_csv('ml-100k/u.data', sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])
ratings.drop('timestamp', axis=1, inplace=True)
ratings.head()

,user_id,item_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [3]:
#perform some preprocessing to encode users and movies as integer indices.

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
ratings['user_id'] = le.fit_transform(ratings['user_id'])
ratings['item_id'] = le.fit_transform(ratings['item_id'])

In [4]:
# prompt: Prepare training and validation data

from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(ratings, test_size=0.2)
train_data.head()

,user_id,item_id,rating
35423,485,761,4
35675,350,310,4
26005,176,10,4
97939,681,721,4
90026,718,97,5


In [5]:
num_users = len(train_data.user_id.unique())
num_movies = len(train_data.item_id.unique())
min_rating = train_data['rating'].min()
max_rating = train_data['rating'].max()
print(     "Number of users: {}, Number of Movies: {}, Min rating: {}, Max rating: {}".format(         num_users, num_movies, min_rating, max_rating     ) )

Number of users: 943, Number of Movies: 1653, Min rating: 1, Max rating: 5


In [6]:
def set_label(data: pd.DataFrame):
    return 1 if data.rating >= 3 else 0

train_data['target'] = train_data.apply(lambda x: 1 if x['rating'] >= 3 else 0, axis=1)
test_data['target'] = test_data.apply(lambda x: 1 if x['rating'] >= 3 else 0, axis=1)

x_train = train_data.drop(columns='target')
y_train = train_data['target']

x_test = test_data.drop(columns='target')
y_test = test_data['target']

In [7]:
class Network:

    def __init__(self, input_size: int, hidden_size: int, output_size: int):
        self.__input_size = input_size
        self.__hidden_size = hidden_size
        self.__output_size = output_size

        self.W1 = np.ones((self.__hidden_size, self.__input_size))
        self.b1 = np.zeros((self.__hidden_size,1))

        self.W2 = np.ones((self.__output_size, self.__hidden_size))
        self.b2 = np.zeros((self.__output_size,1))

        self.z1 = None
        self.z2 = None

    def fit(self, x_input: pd.DataFrame, y_input: pd.DataFrame, iteration=100, learning_rate=0.1):

        y_input_one_hot = pd.get_dummies(y_input).to_numpy()
        x_input = x_input.to_numpy()
        m = y_input_one_hot.shape[0]

        for _ in range(iteration):

            #first layer
            self.z1 = np.matmul(x_input, self.W1.T).T
            self.z1 += self.b1

            #second layer
            a1 = self.leaky_relu(self.z1)
            self.z2 = np.matmul(a1.T, self.W2.T).T
            self.z2 += self.b2

            y_pred = self.softmax(self.z2)

            loss = self.compute_loss(y_real=y_input, y_predicted=y_pred)
            print(np.sum(loss))

            a2 =  self.z2

            #Derivatives
            dz2 = a2 - y_input_one_hot.T
            dw2 = (1/m) * np.matmul(dz2, a1.T)
            db2 = (1/m) * np.sum(dz2)


            da1 = np.matmul(self.W2.T, dz2)
            dz1 = da1 * np.vectorize(self.leaky_relu_derivative)(self.z1)
            dw1 = (1/m) * np.matmul(dz1, x_input)
            db1 = (1./m) * np.sum(dz1)


            # Gradient Descent
            self.W1 = self.W1 - learning_rate * dw1
            self.W2 = self.W2 - learning_rate * dw2

            self.b1 = self.b1 - learning_rate * db1
            self.b2 = self.b2 - learning_rate * db2


    def predict(self):
            pass

    @classmethod
    def leaky_relu(cls, x, alpha=0.01):
        return np.maximum(x*alpha, alpha)

    @classmethod
    def leaky_relu_derivative(cls, x, alpha=0.01):
        return 1.0 if x >= 0.0 else alpha

    @classmethod
    def softmax(cls, x):
        return np.exp(x)/np.sum(np.exp(x))

    @classmethod
    def compute_loss(cls, y_real, y_predicted: pd.DataFrame):
        y_real_one_hot = pd.get_dummies(y_real)

        loss_sum = np.sum(np.multiply(y_real_one_hot.T, np.log(y_predicted)))
        m = y_real_one_hot.shape[0]
        loss = -(1/m) * loss_sum
        return loss

    # def softmax_prime(self, x):
    #     return self.sigmoid(x) * (1 - self.sigmoid(x))

In [8]:
nn = Network(input_size=3, hidden_size=64, output_size=2)

In [9]:
nn.fit(x_train, y_train, iteration=1000, learning_rate=0.05)

C:\Users\acer\AppData\Local\Temp\ipykernel_4136\3571972658.py:72: RuntimeWarning: overflow encountered in exp
  return np.exp(x)/np.sum(np.exp(x))
C:\Users\acer\AppData\Roaming\Python\Python39\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\acer\AppData\Local\Temp\ipykernel_4136\3571972658.py:72: RuntimeWarning: invalid value encountered in divide
  return np.exp(x)/np.sum(np.exp(x))
C:\Users\acer\AppData\Local\Temp\ipykernel_4136\3571972658.py:78: RuntimeWarning: divide by zero encountered in log
  loss_sum = np.sum(np.multiply(y_real_one_hot.T, np.log(y_predicted)))


inf
11.93019648805634
11.930153712770224
11.930110956908774
11.930068220462152
11.930025503420014
11.929982805772466
11.929940127509866
11.929897468621903
11.929854829098907
11.929812208930725
11.92976960810766
11.929727026619588
11.929684464456844
11.929641921609397
11.929599398067143
11.929556893820642
11.929514408859502
11.929471943174118
11.929429496754684
11.929387069591067
11.92934466167343
11.929302272992063
11.929259903537313
11.929217553298798
11.929175222267183
11.929132910432362
11.929090617784373
11.929048344313864
11.929006090010255
11.928963854864467
11.928921638866386
11.928879442006169
11.92883726427416
11.928795105660265
11.928752966154926
11.928710845748412
11.928668744430782
11.92862666219243
11.92858459902348
11.928542554914136
11.928500529854743
11.928458523835332
11.928416536846361
11.928374568878121
11.92833261992068
11.928290689964575
11.9282487789999
11.928206887016952
11.928165014005963
11.928123159957336
11.928081324861356
11.92803950870834
11.927997711488517